In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.3f}'.format)
import warnings
warnings.filterwarnings("ignore")
# 设置中文字体为 SimHei（黑体）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体 SimHei
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

In [43]:
# directory = '../宿州五号机组'
# #输入碳排放数据月文件路径，合并为整年并重采样计算日度数据（碳排放强度均值）
# # List to hold each DataFrame
# dataframes = []
# file = "宿州5号机组-2024年12月.xlsx"
# # Construct full file path
# file_path = os.path.join(directory, file)
# # Read the Excel file into a DataFrame
# df = pd.read_excel(file_path,skiprows=1).iloc[::,2:]
# #转换数据格式 obj——>float 以运算
# df['排放量绩效平均值(g/kwh)'] = pd.to_numeric(df['排放量绩效平均值(g/kwh)'], errors='coerce')
# # Append the DataFrame to the list
# # dataframes.append(df)

# # Concatenate all DataFrames into df for one year 
# # year_df = pd.concat(dataframes, ignore_index=True)



# # # convert date from object type to datetime type 
# df['时间'] = pd.to_datetime(df['时间'])
# # set date to index
# df.set_index('时间', inplace=True)
# # sort data by date
# df = df.sort_index()
# df
# # #选择全年非停机数据
# # year_df_without0 = df[df["有功功率平均值(MW)"] != 0]
# # #计算5mins 间隔的 碳排放强度
# # year_df_without0["碳排放强度"] = ( (year_df_without0["C02浓度平均值(%)"]*44/22.4*10000)/1000*year_df_without0["烟气流量平均值(dNm3/h)"] ) / (1000 * year_df_without0["有功功率平均值(MW)"])
# # year_df_without0
# # # # resample to daily data
# # daily_data = year_df_without0.resample('D').agg({
# #     '有功功率平均值(MW)': 'mean',
# #     '负荷率平均值(%)': 'mean',
# #     'C02浓度平均值(%)': 'mean',
# #     '氧含量平均值(%)': 'mean',
# #     '烟气流量平均值(dNm3/h)': 'mean',
# #     '碳排放量五分钟累计值(t)': 'sum' ,
# #     '排放量绩效平均值(g/kwh)': 'mean',
# #     '碳排放强度': 'mean'
# # })
# # # daily_emssion = daily_data[(daily_data["碳排放强度"] != 0) & (daily_data["碳排放强度"].notnull())]
# # daily_data


In [29]:

#输入碳排放数据月文件路径，合并为整年并重采样计算日度数据（碳排放强度均值）
def get_emission_rate_24(directory):#emssion_data_directory
    # List to hold each DataFrame
    dataframes = []
    # Loop through all files in the directory
    for file in os.listdir(directory): 
        #When an excel file is opened for example by MS excel a hidden temporary file is created in the same directory:~$神皖安庆一号机组23年1月.xlsx
        if not file.startswith("~") and file.endswith('.xlsx'):
            print(file)#打印该路径下读取到的文件名
            # Construct full file path
            file_path = os.path.join(directory, file)
            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path,skiprows=1).iloc[::,2:]
            #转换数据格式 obj——>float 以运算
            
            df['有功功率平均值(MW)'] = pd.to_numeric(df['有功功率平均值(MW)'], errors='coerce')
            df['负荷率平均值(%)'] = pd.to_numeric(df['负荷率平均值(%)'], errors='coerce')
            df['氧含量平均值(%)'] = pd.to_numeric(df['氧含量平均值(%)'], errors='coerce')
            df['烟气流量平均值(dNm3/h)'] = pd.to_numeric(df['烟气流量平均值(dNm3/h)'], errors='coerce')
            df['C02浓度平均值(%)'] = pd.to_numeric(df['C02浓度平均值(%)'], errors='coerce')
            df['碳排放量五分钟累计值(t)'] = pd.to_numeric(df['碳排放量五分钟累计值(t)'], errors='coerce')
            df['排放量绩效平均值(g/kwh)'] = pd.to_numeric(df['排放量绩效平均值(g/kwh)'], errors='coerce')
            # Append the DataFrame to the list
            dataframes.append(df)
            #print(df)
    # Concatenate all DataFrames into df for one year 
    year_df = pd.concat(dataframes, ignore_index=True)
    
    # convert date from object type to datetime type 
    year_df['时间'] = pd.to_datetime(year_df['时间'])
    # set date to index
    year_df.set_index('时间', inplace=True)
    # sort data by date
    year_df = year_df.sort_index()

    #选择全年非停机数据
    year_df_without0 = year_df[year_df["有功功率平均值(MW)"] != 0]
    #计算5mins 间隔的 碳排放强度
    year_df_without0["碳排放强度"] = ( (year_df_without0["C02浓度平均值(%)"]*44/22.4*10000)/1000*year_df_without0["烟气流量平均值(dNm3/h)"] ) / (1000 * year_df_without0["有功功率平均值(MW)"])

    # resample to daily data
    daily_data = year_df_without0.resample('D').agg({
        '有功功率平均值(MW)': 'mean',
        '负荷率平均值(%)': 'mean',
        'C02浓度平均值(%)': 'mean',
        '氧含量平均值(%)': 'mean',
        '烟气流量平均值(dNm3/h)': 'mean',
        '碳排放量五分钟累计值(t)': 'sum' ,
        '排放量绩效平均值(g/kwh)': 'mean',
        '碳排放强度': 'mean'
    })
    daily_emssion = daily_data[(daily_data["碳排放强度"] != 0) & (daily_data["碳排放强度"].notnull())]
    return daily_emssion.reset_index()


In [30]:
directory_5_24 = '../宿州五号机组'
daily_df_5_24 = get_emission_rate_24(directory_5_24)
daily_df_5_24

宿州5号机组-2024年01月.xlsx
宿州5号机组-2024年02月.xlsx
宿州5号机组-2024年03月.xlsx
宿州5号机组-2024年04月.xlsx
宿州5号机组-2024年05月.xlsx
宿州5号机组-2024年06月.xlsx
宿州5号机组-2024年07月.xlsx
宿州5号机组-2024年08月.xlsx
宿州5号机组-2024年09月.xlsx
宿州5号机组-2024年10月.xlsx
宿州5号机组-2024年11月.xlsx
宿州5号机组-2024年12月.xlsx


,时间,有功功率平均值(MW),负荷率平均值(%),C02浓度平均值(%),氧含量平均值(%),烟气流量平均值(dNm3/h),碳排放量五分钟累计值(t),排放量绩效平均值(g/kwh),碳排放强度
0,2024-01-01,234.497,66.998,13.212,6.148,1122057.095,7122.180,1282.882,1261.870
1,2024-01-02,288.363,82.389,13.583,5.501,1249621.623,7753.790,1180.190,1160.852
2,2024-01-03,254.560,72.731,13.235,6.007,1203710.611,7662.320,1269.785,1248.983
3,2024-01-04,219.662,62.761,12.665,6.648,1094618.238,6683.570,1296.388,1275.143
4,2024-01-05,229.784,65.653,12.781,6.287,1088830.584,6692.180,1239.878,1219.569
...,...,...,...,...,...,...,...,...,...
287,2024-12-27,237.397,67.828,11.921,6.522,1062441.768,6101.350,1067.801,1050.314
288,2024-12-28,213.446,60.984,11.756,7.176,1017848.472,5774.550,1131.547,1113.003
289,2024-12-29,200.323,57.235,11.516,7.465,1002339.729,5569.460,1162.336,1143.294
290,2024-12-30,191.417,54.691,11.099,7.827,983735.170,5261.310,1150.750,1131.905


In [33]:
directory_6_24 = '../宿州六号机组'
daily_df_6_24 = get_emission_rate_24(directory_6_24)
daily_df_6_24

宿州6号机组-24年10月.xlsx
宿州6号机组-24年11月.xlsx
宿州6号机组-24年12月.xlsx
宿州6号机组-24年1月.xlsx
宿州6号机组-24年2月.xlsx
宿州6号机组-24年3月.xlsx
宿州6号机组-24年4月.xlsx
宿州6号机组-24年5月.xlsx
宿州6号机组-24年6月.xlsx
宿州6号机组-24年7月.xlsx
宿州6号机组-24年8月.xlsx
宿州6号机组-24年9月.xlsx


,时间,有功功率平均值(MW),负荷率平均值(%),C02浓度平均值(%),氧含量平均值(%),烟气流量平均值(dNm3/h),碳排放量五分钟累计值(t),排放量绩效平均值(g/kwh),碳排放强度
0,2024-01-01,209.154,59.759,12.184,7.322,992301.689,5845.860,1186.949,1167.493
1,2024-01-02,316.260,90.360,13.402,5.804,1318250.616,8062.340,1120.604,1102.242
2,2024-01-03,281.189,80.340,13.023,6.341,1232502.704,7736.850,1160.271,1141.261
3,2024-01-04,214.259,61.218,11.996,7.391,1053047.325,6114.470,1207.378,1187.604
4,2024-01-05,236.064,67.447,12.377,6.975,1083434.431,6488.770,1166.737,1147.622
...,...,...,...,...,...,...,...,...,...
334,2024-12-27,235.190,67.197,13.506,6.546,1211478.513,7856.540,1402.567,1379.598
335,2024-12-28,213.398,60.971,12.797,7.315,1187267.103,7309.340,1444.591,1420.931
336,2024-12-29,204.261,58.360,12.765,7.312,1154698.037,7082.290,1463.311,1439.326
337,2024-12-30,189.504,54.144,12.638,7.212,1088556.184,6626.760,1467.761,1443.715


In [35]:
# 生成完整的日期范围
start_date = daily_df_5_24['时间'].min()  # 获取最小日期
end_date = daily_df_6_24['时间'].max()    # 获取最大日期
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 创建一个包含所有日期的 DataFrame
df_full5 = pd.DataFrame({'时间': date_range})
df_full6 = pd.DataFrame({'时间': date_range})

# 合并原始 DataFrame 和完整日期 DataFrame
df_all5 = pd.merge(df_full5, daily_df_5_24, on='时间', how='left')
df_all6 = pd.merge(df_full6, daily_df_6_24, on='时间', how='left')

# 输出结果查看
df_all5

,时间,有功功率平均值(MW),负荷率平均值(%),C02浓度平均值(%),氧含量平均值(%),烟气流量平均值(dNm3/h),碳排放量五分钟累计值(t),排放量绩效平均值(g/kwh),碳排放强度
0,2024-01-01,234.497,66.998,13.212,6.148,1122057.095,7122.180,1282.882,1261.870
1,2024-01-02,288.363,82.389,13.583,5.501,1249621.623,7753.790,1180.190,1160.852
2,2024-01-03,254.560,72.731,13.235,6.007,1203710.611,7662.320,1269.785,1248.983
3,2024-01-04,219.662,62.761,12.665,6.648,1094618.238,6683.570,1296.388,1275.143
4,2024-01-05,229.784,65.653,12.781,6.287,1088830.584,6692.180,1239.878,1219.569
...,...,...,...,...,...,...,...,...,...
361,2024-12-27,237.397,67.828,11.921,6.522,1062441.768,6101.350,1067.801,1050.314
362,2024-12-28,213.446,60.984,11.756,7.176,1017848.472,5774.550,1131.547,1113.003
363,2024-12-29,200.323,57.235,11.516,7.465,1002339.729,5569.460,1162.336,1143.294
364,2024-12-30,191.417,54.691,11.099,7.827,983735.170,5261.310,1150.750,1131.905


In [36]:
df_all6

,时间,有功功率平均值(MW),负荷率平均值(%),C02浓度平均值(%),氧含量平均值(%),烟气流量平均值(dNm3/h),碳排放量五分钟累计值(t),排放量绩效平均值(g/kwh),碳排放强度
0,2024-01-01,209.154,59.759,12.184,7.322,992301.689,5845.860,1186.949,1167.493
1,2024-01-02,316.260,90.360,13.402,5.804,1318250.616,8062.340,1120.604,1102.242
2,2024-01-03,281.189,80.340,13.023,6.341,1232502.704,7736.850,1160.271,1141.261
3,2024-01-04,214.259,61.218,11.996,7.391,1053047.325,6114.470,1207.378,1187.604
4,2024-01-05,236.064,67.447,12.377,6.975,1083434.431,6488.770,1166.737,1147.622
...,...,...,...,...,...,...,...,...,...
361,2024-12-27,235.190,67.197,13.506,6.546,1211478.513,7856.540,1402.567,1379.598
362,2024-12-28,213.398,60.971,12.797,7.315,1187267.103,7309.340,1444.591,1420.931
363,2024-12-29,204.261,58.360,12.765,7.312,1154698.037,7082.290,1463.311,1439.326
364,2024-12-30,189.504,54.144,12.638,7.212,1088556.184,6626.760,1467.761,1443.715


In [37]:
df_all5.to_excel("../data/宿州五号清洗补充修正.xlsx")
df_all6.to_excel("../data/宿州六号清洗补充修正.xlsx")

In [47]:
df5 = df_all5["碳排放强度"]

In [48]:
df6 = df_all6["碳排放强度"]

In [52]:
import pandas as pd

# 假设 df5 和 df6 已经加载到 DataFrame 中，并且是同样长度的

# 交替合并 df5 和 df6
df_combined = pd.DataFrame({
    'df5_碳排放强度': df5.values,  # df5 的值
    'df6_碳排放强度': df6.values   # df6 的值
})

# 重排索引，使其按照顺序交替排列
df_combined = df_combined.values.flatten()  # 将结果展平成一维数组
df_combined = pd.DataFrame(df_combined, columns=["碳排放强度"])  # 重新转换为 DataFrame

# # 显示合并后的数据
# print(df_combined.head(10))  # 检查结果
df_combined

,碳排放强度
0,1261.870
1,1167.493
2,1160.852
3,1102.242
4,1248.983
...,...
727,1439.326
728,1131.905
729,1443.715
730,1158.427


In [61]:
#与jxd数据728条到12.29对应
df_final = df_combined[0:-4]
df_final

,碳排放强度
0,1261.870
1,1167.493
2,1160.852
3,1102.242
4,1248.983
...,...
723,1379.598
724,1113.003
725,1420.931
726,1143.294


In [55]:
364 *2

728

In [62]:
df_final.to_excel("../data/宿州五号六号碳排放修正数据.xlsx")
#然后手动更改复制  宿州五号六号碳排放修正数据.xlsx 到数据整合.xlsx

In [63]:
#重新去除缺失值

# 读取Excel文件
input_file_path = '../data/数据整合.xlsx'  # 替换为你的Excel文件路径
df = pd.read_excel(input_file_path)

# 删除包含缺失值的列
df_cleaned = df.dropna(axis=0, how='any')

# 将结果保存到新的Excel文件
output_file_path = '../data/数据整合cleaned_new.xlsx'
df_cleaned.to_excel(output_file_path, index=False)
